# Готовий алгоритм та пояснення вибору основної метрики

Збережемо шлях до файлу із даними, значення цільової змінної яких треба передбачити.

In [2]:
path = 'data_to_predict.csv' #insert path to your file in csv or zip format

In [6]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from sklearn.datasets import load_boston
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from sklearn import set_config


dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d') for d in dates]
df = pd.read_csv('dataset.zip', parse_dates=['install_date'], date_parser=dateparse)

df = df.dropna()

df['media_source'] = df['media_source'].apply(lambda t: int(t.replace("SOURCE_", "")))
df['install_date'] = df['install_date'].apply(lambda t: int(t.timestamp()))
df['country_code'] = df['country_code'].apply(lambda t: int(t.replace("COUNTRY_", "")))
df['platform'] = df['platform'].apply(lambda t: 0 if t == 'ios' else 1)

x = df.iloc [:, : -4]
y = np.ravel(df.iloc [:, -1 :], order='C')

x = scale(x)
y = scale(y)

xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.2)

set_config(print_changed_only=False)

regressor = RandomForestRegressor(n_estimators=100)

regressor.fit(xtrain, ytrain)

ypred = regressor.predict(xtest)

mae = mean_absolute_error(ytest, ypred)
mse = mean_squared_error(ytest, ypred)
mape = mean_absolute_percentage_error(ytest, ypred)
rmse = math.sqrt(mse)

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAPE: ", mape)

x_pred = pd.read_csv(path, parse_dates=['install_date'], date_parser=dateparse)

x_pred = x_pred.dropna()

x_pred['media_source'] = df['media_source'].apply(lambda t: int(t.replace("SOURCE_", "")))
x_pred['install_date'] = df['install_date'].apply(lambda t: int(t.timestamp()))
x_pred['country_code'] = df['country_code'].apply(lambda t: int(t.replace("COUNTRY_", "")))
x_pred['platform'] = df['platform'].apply(lambda t: 0 if t == 'ios' else 1)

y_pred = regressor.predict(x_pred)


pd.merge(x_pred, y_pred, how='inner', left_index=True, right_index=True).to_csv('Results', index=False)

KeyboardInterrupt: 

Результати збережуться у файлі з назвою *Results* у поточній директорії.


Основною метрикою для оцінювання якості обраного алгоритму було обрано RMSE (Root Mean Squarre Error), оскільки дана метрика є найбільш розповсюдженою при вирішенні задач даного класу, і на нашу думку, є найбільш репрезентативною.